In [2]:
import shorthand as shnd
s = shnd.Shorthand(
    entry_syntax="shorthand/resources/default_entry_syntax.csv",
    link_syntax="shorthand/resources/default_link_syntax.csv",
    item_separator='__',
    default_entry_prefix='wrk',
    space_char='|',
    na_string_values='!',
    na_node_type='missing',
    syntax_case_sensitive=False
)

parsed = s.parse_text(
    'shorthand/test_data/manual_annotation.shnd',
    skiprows=2,
    comment_char='#'
)
links = parsed.links
links.iloc[60:]
'''
print('       strings', parsed.strings.memory_usage(deep=True).sum()/1000, 'kb')
print('         links', parsed.links.memory_usage(deep=True).sum()/1000, 'kb')
print('resolved links', parsed.resolve_links().memory_usage(deep=True).sum()/1000, 'kb')
'''
entry_type = parsed.id_lookup('link_types', 'entry')
entry_string_ids = parsed.links.loc[parsed.links['link_type_id'] == entry_type, 'tgt_string_id']

parsed.resolve_links().query('src_string.str.contains("nasa")').query('src_node_type != "shorthand_text"').query('tgt_node_type != "shorthand_text"')

parsed.resolve_links().query('link_type == "entry"')

parsed.synthesize_entries('wrk')

7          asmith_bwu__1999__s_bams__101__803__xxx
12          asmith_bwu__1998__s_bams__100__42__yyy
19                  bjones__1975__s_jats__90__1__!
23               bwu__1989__t_long title__!__80__!
26    Some Author__1989__t_A Title With #__!__!__!
46           asmith_bwu__2008__s_bams__110__1__zzz
dtype: object

In [3]:
import pandas as pd

s = pd.DataFrame({
    'item_label': ['1', '2', '3', 'a', 'b'],
    'item_node_type': ['node1', pd.NA, 'node1', 'node2', 'node3'],
    'item_link_type': ['link1', pd.NA, 'link4', 'link2', 'link3'],
    'item_prefix_separator': [pd.NA, '_', pd.NA, pd.NA, pd.NA],
    'item_prefixes': [pd.NA, 'a b', pd.NA, pd.NA, pd.NA]
})

where_item_label_is_position = s['item_label'].str.isdigit()
unprefixed_items = s.loc[
    where_item_label_is_position & s['item_node_type'].notna()
]
prefixed_items = s.loc[
    where_item_label_is_position & s['item_node_type'].isna()
]
optional_items = s.loc[~s['item_label'].str.isdigit()]

s.groupby(by='item_node_type').apply(lambda x: print(x))

  item_label item_node_type item_link_type item_prefix_separator item_prefixes
0          1          node1          link1                  <NA>          <NA>
2          3          node1          link4                  <NA>          <NA>
  item_label item_node_type item_link_type item_prefix_separator item_prefixes
3          a          node2          link2                  <NA>          <NA>
  item_label item_node_type item_link_type item_prefix_separator item_prefixes
4          b          node3          link3                  <NA>          <NA>


""


In [84]:
import pandas as pd
m = pd.DataFrame({'a':[1,2,3,4,5,5], 'b':[pd.NA, '20', '30', pd.NA, 5, 4]}, index=[9,10,11,12,13,14])

result = m.groupby(by='a').apply(lambda x: x.loc[x['b'].astype(pd.Int32Dtype()).idxmin()] if x['b'].notna().any() else None)

pd.concat([m.query('b.isna()'), result.dropna()])


,a,b
9,1.0,<NA>
12,4.0,<NA>
2,2.0,20
3,3.0,30
5,5.0,4


In [28]:
strings = pd.DataFrame({
    'string': pd._testing.rands_array(500, int(1e6)),
    'node_type_id': pd.Series(range(1,6)).sample(int(1e6), replace=True)
})
strings.memory_usage(deep=True).sum()/1e6

573.0

In [3]:
import pandas as pd
pd.Series(dtype='object')

Series([], dtype: object)